In [1]:
import pandas as pd
import sqlite3
from mlxtend.frequent_patterns import apriori,association_rules
import pyodbc

In [2]:
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\harre\DEDS_portfolio\aenc.accdb;')


In [3]:
products = "SELECT * FROM product"
orderdetails = "SELECT * FROM sales_order_item"
producten = pd.read_sql_query(products, conn)
order_details = pd.read_sql_query(orderdetails, conn)
df = pd.merge(order_details, producten, left_on = 'prod_id', right_on='id')
df

C:\Users\harre\AppData\Local\Temp\ipykernel_20396\4262472484.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  producten = pd.read_sql_query(products, conn)
C:\Users\harre\AppData\Local\Temp\ipykernel_20396\4262472484.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_details = pd.read_sql_query(orderdetails, conn)


,id_x,line_id,prod_id,quantity_x,ship_date,id_y,name,description,prod_size,color,quantity_y,unit_price,picture_name,Category
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
1,2006,1,300,48,28-Sep-1996 12:00:00 AM,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
2,2015,1,300,24,06-Oct-1996 12:00:00 AM,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
3,2019,1,300,12,13-Aug-1996 12:00:00 AM,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
4,2030,1,300,24,06-Oct-1996 12:00:00 AM,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2625,3,700,24,09-Nov-1998 12:00:00 AM,700,Shorts,Cotton Shorts,Medium,Black,80,15,shortsb.bmp,Clothes
1099,2635,1,700,72,16-Nov-1998 12:00:00 AM,700,Shorts,Cotton Shorts,Medium,Black,80,15,shortsb.bmp,Clothes
1100,2641,2,700,24,18-Nov-1998 12:00:00 AM,700,Shorts,Cotton Shorts,Medium,Black,80,15,shortsb.bmp,Clothes
1101,2649,1,700,36,26-Nov-1998 12:00:00 AM,700,Shorts,Cotton Shorts,Medium,Black,80,15,shortsb.bmp,Clothes


In [4]:
df_werk = (df[df['Category'] == "Accessories"]
           .groupby(['id_x', 'description'])['quantity_x']
           .sum()
           .unstack()
           .reset_index()
           .fillna(0)
           .set_index('id_x'))
df_werk

description,Cloth Visor,Cotton Cap,Plastic Visor,Wool cap
id_x,,,,
2002,0,24,0,24
2003,12,12,12,0
2009,0,12,0,0
2010,0,0,0,12
2011,36,0,0,0
...,...,...,...,...
2639,0,0,0,36
2641,0,0,36,0
2647,0,0,0,36


In [5]:
def hot_encode(x):
    try:
        x = float(x)  # Converteer x naar float
    except ValueError:
        return 0  # Of een andere waarde voor niet-numerieke invoer

    if x <= 0:
        return 0
    if x >= 1:
        return 1
    return x

# Pas de functie toe op de gehele DataFrame
basket_encoded = df_werk.applymap(hot_encode)
df = basket_encoded
df

description,Cloth Visor,Cotton Cap,Plastic Visor,Wool cap
id_x,,,,
2002,0,1,0,1
2003,1,1,1,0
2009,0,1,0,0
2010,0,0,0,1
2011,1,0,0,0
...,...,...,...,...
2639,0,0,0,1
2641,0,0,1,0
2647,0,0,0,1


In [6]:
# Deze regel gebruikt het Apriori-algoritme om frequente itemsets te vinden in de DataFrame df.
# min_support = 0.01 geeft de minimumsupportdrempel aan waaraan een itemset moet voldoen om als frequent te worden beschouwd.
# use_colnames = True geeft aan dat kolomnamen moeten worden gebruikt in de uitvoer DataFrame
frq_items = apriori(df, min_support = 0.02, use_colnames = True)

# Deze regel genereert associatieregels uit de frequente itemsets frq_items.
# De parameter metric geeft aan welke maatstaf moet worden gebruikt om de gegenereerde regels te evalueren, hier is "lift" gekozen.
# min_threshold = 1 stelt de minimumdrempel in voor de evaluatiemaatstaf. In dit geval worden regels met een lift groter dan of gelijk aan 1 beschouwd.
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

# Deze regel sorteert de gegenereerde associatieregels op twee criteria: confidence en lift.
# ascending =[False, False] geeft aan dat de sorteer volgorde voor zowel confidence als lift aflopend is (van hoog naar laag).
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules


C:\Users\harre\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9,"(Cloth Visor, Wool cap)",(Cotton Cap),0.042105,0.473684,0.035088,0.833333,1.759259,0.015143,3.157895,0.450549
5,"(Cotton Cap, Plastic Visor)",(Cloth Visor),0.042105,0.368421,0.031579,0.750000,2.035714,0.016066,2.526316,0.531136
12,"(Wool cap, Plastic Visor)",(Cotton Cap),0.028070,0.473684,0.021053,0.750000,1.583333,0.007756,2.105263,0.379061
2,(Wool cap),(Cotton Cap),0.375439,0.473684,0.256140,0.682243,1.440291,0.078301,1.656347,0.489457
1,(Plastic Visor),(Cloth Visor),0.350877,0.368421,0.235088,0.670000,1.818571,0.105817,1.913876,0.693425
8,"(Cloth Visor, Cotton Cap)",(Wool cap),0.052632,0.375439,0.035088,0.666667,1.775701,0.015328,1.873684,0.461111
0,(Cloth Visor),(Plastic Visor),0.368421,0.350877,0.235088,0.638095,1.818571,0.105817,1.793629,0.712687
4,"(Cloth Visor, Cotton Cap)",(Plastic Visor),0.052632,0.350877,0.031579,0.600000,1.710000,0.013112,1.622807,0.438272
3,(Cotton Cap),(Wool cap),0.473684,0.375439,0.256140,0.540741,1.440291,0.078301,1.359932,0.580822
13,"(Cotton Cap, Plastic Visor)",(Wool cap),0.042105,0.375439,0.021053,0.500000,1.331776,0.005245,1.249123,0.260073
